# Aim of this code

Print table of representative ISM conditions, calculate everything. 

This code requires "calc_LTE_alpha_CO_CI_new.ipynb".


In [1]:
import os, sys, re
import numpy as np
from astropy.table import Table
from collections import OrderedDict

In [2]:
# Load functions from JupyterNotebooks

#%run calc_LTE_alpha_CO_CI_new.ipynb # this will be loaded in the line below

%run calc_NonLTE_RADEX_plot_alpha_CI10_correction_matrix.ipynb
%run calc_NonLTE_RADEX_plot_alpha_CO10_correction_matrix.ipynb

assert len(list_of_I_CO10_range_for_N_repISM) > 0
assert len(list_of_d_V_for_N_repISM) > 0
assert len(list_of_R_CI10CO21_for_N_repISM) > 0
assert len(CO10_T_only_popt) > 0
assert func_T_only is not None
assert func_T_NLCorr_alphaCO10 is not None


list_of_alphaCO10_for_N_repISM [2.574, 1.471, 0.692, 0.654, 0.753, 0.573, 0.525]
list_of_Sigma_H2_for_N_repISM [102.941, 220.588, 294.118, 588.235, 640.414, 687.798, 1102.941]
list_of_Sigma_mol_for_N_repISM [140.000, 300.000, 400.000, 800.000, 870.964, 935.406, 1500.000]
list_of_N_H2_for_N_repISM [6.424e+21, 1.376e+22, 1.835e+22, 3.671e+22, 3.996e+22, 4.292e+22, 6.882e+22]
list_of_N_CO_for_N_repISM [6.424e+17, 1.376e+18, 1.835e+18, 3.671e+18, 3.996e+18, 4.292e+18, 6.882e+18]
list_of_N_CI_for_N_repISM [1.285e+17, 2.753e+17, 3.671e+17, 7.341e+17, 7.992e+17, 8.584e+17, 1.376e+18]
list_of_N_CO_d_V_for_N_repISM [2.141e+17, 1.376e+17, 7.341e+16, 7.341e+16, 9.990e+16, 6.131e+16, 7.647e+16]
Loading 'out_LTE_alphaCO_alphaCI_new_QE_term_popt.json'
CO10_C_coeff = 0.019154525467252038 # s solMass / (K km pc2)
CO10_QE_popt = [1.1841972425580587, 1.0501500500798997, 0.13536555224402363, 3.0465464845741184]
CO21_C_coeff = 0.004791830144641124 # s solMass / (K km pc2)
CO21_QE_popt = [0.320706452865248

In [3]:
# Set basic parameters for our representative ISM conditions

# below are the same as in "calc_LTE_R_CI10CO21_varying_N_CO_T_kin.ipynb"
list_of_T_kin_for_N_repISM = np.array([15., 20., 25., 50., 50., 100., 100.]) # T_kin in K

# below are the same as in "calc_NLCorr_alphaCO_alphaCI_RCICO.ipynb"
list_of_n_H2_for_N_repISM = 10**np.array([2.5, 2.5, 3.0, 3.0, 3.0, 3.0, 3.0]) # n_H2 in cm^-3 for repISM

# below are from the calculation of "calc_NLCorr_alphaCO_alphaCI_RCICO.ipynb"
# repISM 1 : alphaCO10^NoHeLTE = 2.78, NLTE = 3.11 (w/He 4.23), NCO/Dv = 2.00e+17
# repISM 2 : alphaCO10^NoHeLTE = 1.86, NLTE = 2.16 (w/He 2.93), NCO/Dv = 1.78e+17
# repISM 3 : alphaCO10^NoHeLTE = 1.41, NLTE = 0.98 (w/He 1.33), NCO/Dv = 7.94e+16
# repISM 4 : alphaCO10^NoHeLTE = 0.84, NLTE = 0.47 (w/He 0.64), NCO/Dv = 7.08e+16
# repISM 5 : alphaCO10^NoHeLTE = 0.84, NLTE = 0.58 (w/He 0.78), NCO/Dv = 1.00e+17
# repISM 6 : alphaCO10^NoHeLTE = 0.95, NLTE = 0.36 (w/He 0.50), NCO/Dv = 6.31e+16
# repISM 7 : alphaCO10^NoHeLTE = 0.95, NLTE = 0.44 (w/He 0.59), NCO/Dv = 8.91e+16
list_of_N_CO_d_V_for_N_repISM = np.array([2.00e+17, 1.78e+17, 7.94e+16, 7.08e+16, 
                                          1.00e+17, 6.31e+16, 8.91e+16])

# below are assumed based on our paper I
list_of_CO_plus_CI_abundance_for_N_repISM = np.array([1.2e-4]*7) # [(CO+CI)/H2]
list_of_X_CI_CO_for_N_repISM = np.array([0.3, 0.3, 0.4, 1.0, 0.2, 0.6, 2.0]) # [CI/CO]
list_of_CO_abundance_for_N_repISM = (1.0/(1.0+list_of_X_CI_CO_for_N_repISM)) \
                                    * list_of_CO_plus_CI_abundance_for_N_repISM
list_of_CI_abundance_for_N_repISM = (1.0/(1.0+1.0/list_of_X_CI_CO_for_N_repISM)) \
                                    * list_of_CO_plus_CI_abundance_for_N_repISM
print('list_of_CO_plus_CI_abundance_for_N_repISM', list_of_CO_plus_CI_abundance_for_N_repISM)
print('list_of_CO_abundance_for_N_repISM', list_of_CO_abundance_for_N_repISM)
print('list_of_CI_abundance_for_N_repISM', list_of_CI_abundance_for_N_repISM)

# Or for simplicity we just assume Galactic NCODV
#list_of_N_CO_d_V_for_N_repISM = np.array([2.00e+17]*7)


list_of_CO_plus_CI_abundance_for_N_repISM [0.00012 0.00012 0.00012 0.00012 0.00012 0.00012 0.00012]
list_of_CO_abundance_for_N_repISM [9.23076923e-05 9.23076923e-05 8.57142857e-05 6.00000000e-05
 1.00000000e-04 7.50000000e-05 4.00000000e-05]
list_of_CI_abundance_for_N_repISM [2.76923077e-05 2.76923077e-05 3.42857143e-05 6.00000000e-05
 2.00000000e-05 4.50000000e-05 8.00000000e-05]


In [4]:
# Apply T-only formular for CO10

alpha_CO10_T_only = func_alpha_CO(list_of_T_kin_for_N_repISM, *CO10_T_only_popt)

print('alpha_CO10_T_only:', repr(alpha_CO10_T_only))

alpha_CO10_T_only: array([2.79828972, 1.99005094, 1.54490075, 0.86543872, 0.86543872,
       0.91093672, 0.91093672])


In [5]:
# Apply NLCorr formular for CO10

# using constant N_CO/D_v

alpha_CO10_T_NLCorr1 = np.array([
    func_T_NLCorr_alphaCO10(t, a, N_CO_d_V = 2e17, n_H2 = nh) \
        for t, a, nh in list(zip(list_of_T_kin_for_N_repISM, 
                                 list_of_CO_abundance_for_N_repISM, 
                                 list_of_n_H2_for_N_repISM))
])

print('alpha_CO10_T_NLCorr1:', repr(alpha_CO10_T_NLCorr1), 
      '(approximated NLCorr function, assuming realistic N_CO/D_v)')

# using assumed N_CO/D_v

alpha_CO10_T_NLCorr2 = np.array([
    func_T_NLCorr_alphaCO10(t, a, N_CO_d_V = nv, n_H2 = nh) \
        for t, a, nv, nh in list(zip(list_of_T_kin_for_N_repISM, 
                                     list_of_CO_abundance_for_N_repISM, 
                                     list_of_N_CO_d_V_for_N_repISM, 
                                     list_of_n_H2_for_N_repISM))
])

print('alpha_CO10_T_NLCorr2:', repr(alpha_CO10_T_NLCorr2), 
      '(approximated NLCorr function, assuming constant N_CO/D_v)')

# using precise interpolation

def func_alphaCO10_T_NLCorr_matrix(T_kin, CO_abundance, N_CO_d_V, n_H2):
    return func_T_only(list_of_T_kin_for_N_repISM, *CO10_T_only_popt) \
           * (1e-4) / CO_abundance \
           * interpolate_NLCorr(n_H2, N_CO_d_V, T_kin, 'out_NLCorr_matrix_alphaCO10.fits')

alpha_CO10_T_NLCorr3 = func_alphaCO10_T_NLCorr_matrix(list_of_T_kin_for_N_repISM, 
                                 list_of_CO_abundance_for_N_repISM, 
                                 list_of_N_CO_d_V_for_N_repISM, 
                                 list_of_n_H2_for_N_repISM)

print('alpha_CO10_T_NLCorr3:', repr(alpha_CO10_T_NLCorr3), 
      '(precise NLCorr interpolation)')



alpha_CO10_T_NLCorr1: array([3.6570723 , 2.50640627, 1.8062275 , 1.69400319, 1.01640191,
       1.15134416, 2.1587703 ]) (approximated NLCorr function, assuming realistic N_CO/D_v)
alpha_CO10_T_NLCorr2: array([3.6570723 , 2.23070919, 0.71769818, 0.69189864, 0.55911673,
       0.7001403 , 1.52334246]) (approximated NLCorr function, assuming constant N_CO/D_v)
alpha_CO10_T_NLCorr3: array([3.38062479, 2.33942436, 0.86923451, 0.78959262, 0.57672778,
       0.4861606 , 1.09127029]) (precise NLCorr interpolation)


In [6]:
# Apply T-only formular for CI10

alpha_CI10_T_only = func_alpha_CI(list_of_T_kin_for_N_repISM, *CI10_T_only_popt)

print('alpha_CI10_T_only:', repr(alpha_CI10_T_only))

alpha_CI10_T_only: array([15.03789309, 12.55852836, 11.86916873, 11.65237762, 11.65237762,
       12.25369895, 12.25369895])


In [7]:
# Apply NLCorr formular for CI10

# using constant N_CO/D_v

alpha_CI10_T_NLCorr1 = np.array([
    func_T_NLCorr_alphaCI10(t, a, N_CO_d_V = 2e17, n_H2 = nh) \
        for t, a, nh in list(zip(list_of_T_kin_for_N_repISM, 
                                 list_of_CI_abundance_for_N_repISM, 
                                 list_of_n_H2_for_N_repISM))
])

print('alpha_CI10_T_NLCorr1:', repr(alpha_CI10_T_NLCorr1), 
      '(approximated NLCorr function, assuming realistic N_CI/D_v)')

# using assumed N_CO/D_v

alpha_CI10_T_NLCorr2 = np.array([
    func_T_NLCorr_alphaCI10(t, a, N_CO_d_V = nv, n_H2 = nh) \
        for t, a, nv, nh in list(zip(list_of_T_kin_for_N_repISM, 
                                     list_of_CI_abundance_for_N_repISM, 
                                     list_of_N_CO_d_V_for_N_repISM, 
                                     list_of_n_H2_for_N_repISM))
])

print('alpha_CI10_T_NLCorr2:', repr(alpha_CI10_T_NLCorr2), 
      '(approximated NLCorr function, assuming constant N_CI/D_v)')

# using precise interpolation

def func_alphaCI10_T_NLCorr_matrix(T_kin, CI_abundance, N_CI_d_V, n_H2):
    return func_T_only(list_of_T_kin_for_N_repISM, *CI10_T_only_popt) \
           * (2e-5) / CI_abundance \
           * interpolate_NLCorr(n_H2, N_CI_d_V, T_kin, 'out_NLCorr_matrix_alphaCI10.fits')

alpha_CI10_T_NLCorr3 = func_alphaCI10_T_NLCorr_matrix(list_of_T_kin_for_N_repISM, 
                                 list_of_CI_abundance_for_N_repISM, 
                                 list_of_N_CO_d_V_for_N_repISM, 
                                 list_of_n_H2_for_N_repISM)

print('alpha_CI10_T_NLCorr3:', repr(alpha_CI10_T_NLCorr3), 
      '(precise NLCorr interpolation)')



alpha_CI10_T_NLCorr1: array([27.31123532, 18.69360642,  8.38617981,  4.08987366, 12.26962099,
        5.47907012,  3.08197694]) (approximated NLCorr function, assuming realistic N_CI/D_v)
alpha_CI10_T_NLCorr2: array([27.31123532, 18.69360642,  8.38617981,  4.08987366, 12.26962099,
        5.47907012,  3.08197694]) (approximated NLCorr function, assuming constant N_CI/D_v)
alpha_CI10_T_NLCorr3: array([38.194885  , 24.37186997,  8.94970895,  3.82973634, 11.98949661,
        4.74073913,  2.73917971]) (precise NLCorr interpolation)


In [8]:
# Build the table

table_dict = OrderedDict()
table_dict['ISM condition assumptions'] = [
    r'\multirow[t]{4}{2.3cm}{[1] Galactic clouds}', 
    r'\multirow[t]{4}{2.3cm}{[2] NGC~3627 SF disk}', 
    r'\multirow[t]{4}{2.3cm}{[3] NGC~3627 SF bar-end}', 
    r'\multirow[t]{4}{2.3cm}{[4] NGC~1808/{\allowbreak}7469 SB disk}', 
    r'\multirow[t]{4}{2.3cm}{[5] NGC~4321 LINER nuc}', 
    r'\multirow[t]{4}{2.3cm}{[6] NGC~3627 lo-AGN nuc}', 
    r'\multirow[t]{4}{2.3cm}{[7] NGC~7469 hi-AGN nuc}']
table_dict['I_CO10'] = ['{:g}--{:g}'.format(t[0], t[1]) for t in list_of_I_CO10_range_for_N_repISM]
table_dict['Delta_v'] = list_of_d_V_for_N_repISM
table_dict['R_CI10CO21'] = list_of_R_CI10CO21_for_N_repISM
table_dict['T_kin'] = ['{:g}'.format(
    t) for t in list_of_T_kin_for_N_repISM]
table_dict['CICO_H_2_1e4'] = ['{:g} ({:.1f})'.format(
    a*1e4, r) for a, r in list(zip(list_of_CO_plus_CI_abundance_for_N_repISM, 
                                   list_of_X_CI_CO_for_N_repISM))]

table_dict['alphaCO_T_only'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CO10_T_only]
table_dict['alphaCI_T_only'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CI10_T_only]
table_dict['N_H2_T_only'] = ['{:.2f}--{:.2f}'.format(
    np.log10(t[0]*a*0.624e20), np.log10(t[1]*a*0.624e20)) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_only))]
table_dict['Sig_H2_T_only'] = ['{:.0f}--{:.0f}'.format(
    t[0]*a*1.36, t[1]*a*1.36) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_only))]

table_dict['alphaCO_T_NLCorr1'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CO10_T_NLCorr1]
table_dict['alphaCI_T_NLCorr1'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CI10_T_NLCorr1]
table_dict['N_H2_T_NLCorr1'] = ['{:.2f}--{:.2f}'.format(
    np.log10(t[0]*a*0.624e20), np.log10(t[1]*a*0.624e20)) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr1))]
table_dict['Sig_H2_T_NLCorr1'] = ['{:.0f}--{:.0f}'.format(
    t[0]*a*1.36, t[1]*a*1.36) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr1))]

table_dict['alphaCO_T_NLCorr2'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CO10_T_NLCorr2]
table_dict['alphaCI_T_NLCorr2'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CI10_T_NLCorr2]
table_dict['N_H2_T_NLCorr2'] = ['{:.2f}--{:.2f}'.format(
    np.log10(t[0]*a*0.624e20), np.log10(t[1]*a*0.624e20)) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr2))]
table_dict['Sig_H2_T_NLCorr2'] = ['{:.0f}--{:.0f}'.format(
    t[0]*a*1.36, t[1]*a*1.36) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr2))]

table_dict['alphaCO_T_NLCorr3'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CO10_T_NLCorr3]
table_dict['alphaCI_T_NLCorr3'] = ['{:.2f}'.format(
    a*1.36) for a in alpha_CI10_T_NLCorr3]
table_dict['N_H2_T_NLCorr3'] = ['{:.2f}--{:.2f}'.format(
    np.log10(t[0]*a*0.624e20), np.log10(t[1]*a*0.624e20)) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr3))]
table_dict['Sig_H2_T_NLCorr3'] = ['{:.0f}--{:.0f}'.format(
    t[0]*a*1.36, t[1]*a*1.36) for t,a in \
        list(zip(list_of_I_CO10_range_for_N_repISM, alpha_CO10_T_NLCorr3))]

table_data = Table(table_dict)

display(table_data)


ISM condition assumptions,I_CO10,Delta_v,R_CI10CO21,T_kin,CICO_H_2_1e4,alphaCO_T_only,alphaCI_T_only,N_H2_T_only,Sig_H2_T_only,alphaCO_T_NLCorr1,alphaCI_T_NLCorr1,N_H2_T_NLCorr1,Sig_H2_T_NLCorr1,alphaCO_T_NLCorr2,alphaCI_T_NLCorr2,N_H2_T_NLCorr2,Sig_H2_T_NLCorr2,alphaCO_T_NLCorr3,alphaCI_T_NLCorr3,N_H2_T_NLCorr3,Sig_H2_T_NLCorr3
str62,str10,float64,float64,str3,str9,str4,str5,str12,str10,str4,str5,str12,str10,str4,str5,str12,str10,str4,str5,str12,str10
\multirow[t]{4}{2.3cm}{[1] Galactic clouds},30--50,3.0,0.1,15,1.2 (0.3),3.81,20.45,21.72--21.94,114--190,4.97,37.14,21.84--22.06,149--249,4.97,37.14,21.84--22.06,149--249,4.60,51.95,21.80--22.02,138--230
\multirow[t]{4}{2.3cm}{[2] NGC~3627 SF disk},100--200,10.0,0.1,20,1.2 (0.3),2.71,17.08,22.09--22.40,271--541,3.41,25.42,22.19--22.50,341--682,3.03,25.42,22.14--22.44,303--607,3.18,33.15,22.16--22.47,318--636
\multirow[t]{4}{2.3cm}{[3] NGC~3627 SF bar-end},350--500,25.0,0.1,25,1.2 (0.4),2.10,16.14,22.53--22.68,735--1051,2.46,11.41,22.60--22.75,860--1228,0.98,11.41,22.20--22.35,342--488,1.18,12.17,22.28--22.43,414--591
\multirow[t]{4}{2.3cm}{[4] NGC~1808/{\allowbreak}7469 SB disk},800--1000,50.0,0.2,50,1.2 (1.0),1.18,15.85,22.64--22.73,942--1177,2.30,5.56,22.93--23.02,1843--2304,0.94,5.56,22.54--22.64,753--941,1.07,5.21,22.60--22.69,859--1074
\multirow[t]{4}{2.3cm}{[5] NGC~4321 LINER nuc},700--1000,40.0,0.06,50,1.2 (0.2),1.18,15.85,22.58--22.73,824--1177,1.38,16.69,22.65--22.80,968--1382,0.76,16.69,22.39--22.54,532--760,0.78,16.31,22.40--22.56,549--784
\multirow[t]{4}{2.3cm}{[6] NGC~3627 lo-AGN nuc},1000--1400,70.0,0.1,100,1.2 (0.6),1.24,16.67,22.75--22.90,1239--1734,1.57,7.45,22.86--23.00,1566--2192,0.95,7.45,22.64--22.79,952--1333,0.66,6.45,22.48--22.63,661--926
\multirow[t]{4}{2.3cm}{[7] NGC~7469 hi-AGN nuc},2000--2200,90.0,0.5,100,1.2 (2.0),1.24,16.67,23.06--23.10,2478--2726,2.94,4.19,23.43--23.47,5872--6459,2.07,4.19,23.28--23.32,4143--4558,1.48,3.73,23.13--23.18,2968--3265


In [9]:
from astropy.io import ascii
table_data2 = OrderedDict()
table_data2['ISM condition assumptions'] = ['']*(len(table_data)*4)
table_data2['I_CO10'] = ['']*(len(table_data)*4)
table_data2['Delta_v'] = ['']*(len(table_data)*4)
table_data2['R_CI10CO21'] = ['']*(len(table_data)*4)
table_data2['T_kin'] = ['']*(len(table_data)*4)
table_data2['CICO_H_2_1e4'] = ['']*(len(table_data)*4)
table_data2['alphaCO'] = ['']*(len(table_data)*4)
table_data2['alphaCI'] = ['']*(len(table_data)*4)
table_data2['type'] = ['']*(len(table_data)*4)
table_data2['Sig_H2'] = ['']*(len(table_data)*4)
table_data2['NCO_DV'] = ['']*(len(table_data)*4)
# table_data2['Ref'] = ['']*(len(table_data)*4)
for i in range(len(table_data)):
    table_data2['ISM condition assumptions'][4*i] = table_data['ISM condition assumptions'][i]
    if i > 0:
        table_data2['ISM condition assumptions'][4*i] = '[1ex]' + '\n' + \
            table_data2['ISM condition assumptions'][4*i]
    table_data2['I_CO10'][4*i] = table_data['I_CO10'][i]
    table_data2['Delta_v'][4*i] = table_data['Delta_v'][i]
    table_data2['R_CI10CO21'][4*i] = table_data['R_CI10CO21'][i]
    table_data2['T_kin'][4*i] = table_data['T_kin'][i]
    table_data2['CICO_H_2_1e4'][4*i] = table_data['CICO_H_2_1e4'][i].split()[0]+'e-4'
    table_data2['CICO_H_2_1e4'][4*i+1] = table_data['CICO_H_2_1e4'][i].split()[1]
    table_data2['alphaCO'][4*i] = table_data['alphaCO_T_only'][i]
    table_data2['alphaCO'][4*i+1] = table_data['alphaCO_T_NLCorr1'][i]
    table_data2['alphaCO'][4*i+2] = table_data['alphaCO_T_NLCorr2'][i]
    table_data2['alphaCO'][4*i+3] = table_data['alphaCO_T_NLCorr3'][i]
    table_data2['alphaCI'][4*i] = table_data['alphaCI_T_only'][i]
    table_data2['alphaCI'][4*i+1] = table_data['alphaCI_T_NLCorr1'][i]
    table_data2['alphaCI'][4*i+2] = table_data['alphaCI_T_NLCorr2'][i]
    table_data2['alphaCI'][4*i+3] = table_data['alphaCI_T_NLCorr3'][i]
    table_data2['Sig_H2'][4*i] = table_data['Sig_H2_T_only'][i]
    table_data2['Sig_H2'][4*i+1] = table_data['Sig_H2_T_NLCorr1'][i]
    table_data2['Sig_H2'][4*i+2] = table_data['Sig_H2_T_NLCorr2'][i]
    table_data2['Sig_H2'][4*i+3] = table_data['Sig_H2_T_NLCorr3'][i]
    table_data2['NCO_DV'][4*i] = '---'
    table_data2['NCO_DV'][4*i+1] = '{:.2g}'.format(2e17/1e17)+'e17'
    table_data2['NCO_DV'][4*i+2] = '{:.2g}'.format(list_of_N_CO_d_V_for_N_repISM[i]/1e17)+'e17'
    table_data2['NCO_DV'][4*i+3] = '{:.2g}'.format(list_of_N_CO_d_V_for_N_repISM[i]/1e17)+'e17'
    table_data2['type'][4*i] = 'T-only'
    table_data2['type'][4*i+1] = 'NLCorr1'
    table_data2['type'][4*i+2] = 'NLCorr2'
    table_data2['type'][4*i+3] = 'NLCorr3'
# table_data2['Ref'][4*0] = '[1,2,3]'
# table_data2['Ref'][4*1] = '[4,5]'
# table_data2['Ref'][4*2] = '[4,5]'
# table_data2['Ref'][4*3] = '[4,6,7]'
# table_data2['Ref'][4*4] = '[4,5]'
# table_data2['Ref'][4*5] = '[4,5]'
# table_data2['Ref'][4*6] = '[4,7]'
ascii.write(table_data2, Writer=ascii.Latex,
            #latexdict = {'units': {'mass': 'kg', 'speed': 'km/h'}},
           )

\begin{table}
\begin{tabular}{ccccccccccc}
ISM condition assumptions & I_CO10 & Delta_v & R_CI10CO21 & T_kin & CICO_H_2_1e4 & alphaCO & alphaCI & type & Sig_H2 & NCO_DV \\
\multirow[t]{4}{2.3cm}{[1] Galactic clouds} & 30--50 & 3.0 & 0.1 & 15 & 1.2e-4 & 3.81 & 20.45 & T-only & 114--190 & --- \\
 &  &  &  &  & (0.3) & 4.97 & 37.14 & NLCorr1 & 149--249 & 2e17 \\
 &  &  &  &  &  & 4.97 & 37.14 & NLCorr2 & 149--249 & 2e17 \\
 &  &  &  &  &  & 4.60 & 51.95 & NLCorr3 & 138--230 & 2e17 \\
[1ex]
\multirow[t]{4}{2.3cm}{[2] NGC~3627 SF disk} & 100--200 & 10.0 & 0.1 & 20 & 1.2e-4 & 2.71 & 17.08 & T-only & 271--541 & --- \\
 &  &  &  &  & (0.3) & 3.41 & 25.42 & NLCorr1 & 341--682 & 2e17 \\
 &  &  &  &  &  & 3.03 & 25.42 & NLCorr2 & 303--607 & 1.8e17 \\
 &  &  &  &  &  & 3.18 & 33.15 & NLCorr3 & 318--636 & 1.8e17 \\
[1ex]
\multirow[t]{4}{2.3cm}{[3] NGC~3627 SF bar-end} & 350--500 & 25.0 & 0.1 & 25 & 1.2e-4 & 2.10 & 16.14 & T-only & 735--1051 & --- \\
 &  &  &  &  & (0.4) & 2.46 & 11.41 & NLCorr1 & 8